In [106]:
from transformers import pipeline
classifier = pipeline("text-classification", model="shahiryar/tt_abstract_classifier")

2024-02-15 11:45:59.010663: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-02-15 11:45:59.010711: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-02-15 11:45:59.010724: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-02-15 11:45:59.010822: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-15 11:45:59.010893: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the mode

In [5]:
tokenizer = classifier.tokenizer
model = classifier.model

In [6]:
#preprocessing function to tokenize text and truncate sequences to be no longer than DistilBERT’s maximum input length:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True) 

In [39]:
#preprocess the log data
import pandas as pd
log_df = pd.read_csv('../dataset/log.csv')
log_df = log_df[["abstract", "label"]]
log_df.columns = ['text', 'label']
log_df.head()

,text,label


In [40]:
#collect previous dataset
labeled_dataset_path = '../dataset/Paper Abstracts and Categorization Shahiryar.csv'
labled_df = pd.read_csv(labeled_dataset_path)
labled_df.head()

df_clean = labled_df[["Paper ID", "Title", "Abstract", "TT", "TC"]]
to_lower = lambda s: s.lower() if s==s else ""
strip = lambda s: s.strip() if s==s else ""
df_clean.Title = df_clean["Title"].map(to_lower)
df_clean.Abstract = df_clean.Abstract.map(to_lower)
df_clean.TT = df_clean.TT.map(strip)
df_clean.TC = df_clean.TC.map(strip)

df_clean = df_clean.iloc[:-3, :-1]
df_clean['text'] = df_clean['Title']+" "+df_clean['Abstract']
df_clean['label'] = df_clean['TT']
df_clean = df_clean.iloc[:, -2:]
df_clean.head()

/var/folders/_v/dyxdn97j4931qv74z2fkldxr0000gn/T/ipykernel_948/3182036256.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.Title = df_clean["Title"].map(to_lower)
/var/folders/_v/dyxdn97j4931qv74z2fkldxr0000gn/T/ipykernel_948/3182036256.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.Abstract = df_clean.Abstract.map(to_lower)
/var/folders/_v/dyxdn97j4931qv74z2fkldxr0000gn/T/ipykernel_948/3182036256.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

,text,label
0,"the knowledge-based view, nested heterogeneity...",TT5
1,a multidimensional conceptualization of enviro...,TT5
2,an attention-based view of real options reason...,TT5
3,the dark side of organizational paradoxes: the...,TT5
4,exploring intuition and its role in managerial...,TT5


In [41]:
df = pd.concat([df_clean, log_df])
df.drop_duplicates(subset=["text"], inplace=True)

In [42]:
classes = sorted(list(df.label.value_counts().index))
NUM_CLASSES = len(classes)
id2label = { i: el for i, el in enumerate(classes)}
label2id = { el: i for i, el in enumerate(classes)}

In [43]:
label2id

{'TT1': 0, 'TT5': 1}

In [7]:
df.label = df.label.replace(label2id)

/var/folders/_v/dyxdn97j4931qv74z2fkldxr0000gn/T/ipykernel_948/890478537.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.label = df.label.replace(label2id)


In [9]:
df.reset_index(drop=True, inplace=True)

In [10]:
df.head()

,text,label
0,"the knowledge-based view, nested heterogeneity...",1
1,a multidimensional conceptualization of enviro...,1
2,an attention-based view of real options reason...,1
3,the dark side of organizational paradoxes: the...,1
4,exploring intuition and its role in managerial...,1


In [11]:
from datasets import Dataset

/Users/shahiryarsaleem/Documents/Projects/TT Classifier/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
train_dataset = Dataset.from_pandas(df, preserve_index=False)

In [13]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 37
})

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 35.5kB/s]
config.json: 100%|██████████| 483/483 [00:00<00:00, 4.72MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:01<00:00, 143kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 1.47MB/s]


In [15]:
#preprocessing function to tokenize text and truncate sequences to be no longer than DistilBERT’s maximum input length:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [16]:
#Apply the preprocessing function over the entire dataset, use 🤗 Datasets map function
tokenized_papers = train_dataset.map(preprocess_function)

Map: 100%|██████████| 37/37 [00:00<00:00, 1356.30 examples/s]


In [17]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [19]:
import evaluate

accuracy = evaluate.load("accuracy")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
#create a function that passes your predictions and labels to compute to calculate the accuracy:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [21]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 7
num_epochs = 15
batches_per_epoch = len(tokenized_papers) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

2024-02-17 09:01:22.911672: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-02-17 09:01:22.911700: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-02-17 09:01:22.911708: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-02-17 09:01:22.911767: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-17 09:01:22.911812: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [23]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=NUM_CLASSES, id2label=id2label, label2id=label2id
)

model.safetensors:  55%|█████▍    | 147M/268M [01:35<01:02, 1.93MB/s] Error while downloading from https://cdn-lfs.huggingface.co/distilbert-base-uncased/5e3f1108e3cb34ee048634875d8482665b65ac713291a7e32396fb18f6ff0063?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1708401325&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwODQwMTMyNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9kaXN0aWxiZXJ0LWJhc2UtdW5jYXNlZC81ZTNmMTEwOGUzY2IzNGVlMDQ4NjM0ODc1ZDg0ODI2NjViNjVhYzcxMzI5MWE3ZTMyMzk2ZmIxOGY2ZmYwMDYzP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=iiM0komK%7E943Are2%7E2bwN4GIr0cyOO57w7D83XquLN4CAMoyGi21FxEt-qOlj8FTW4ubUHau-Crwg9J6bKgx-u-INPpDb94Rz7U9unTmtmko1NBo-j7e%7E8FJ5RKz6X5%7E9oinerRQ78ZciyR9UeH09V2P8E42%7EP-k0sMjsvM%7EVutVsuP6cxpeRC5AqFxN7xWVx9YRA7Q5VaYlNRaNrpY2x%7Eaj9FfLzAMt9mzmb087xPtFDqLE1Q7FwNuWpD9sx7b-eFOyAiLQv3eYM4QTmlFjWdhMvLLIl

In [24]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_papers,
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

In [25]:
import tensorflow as tf
#==============================
#Compile Text Classifier Model
#==============================
model.compile(optimizer=optimizer)  # No loss argument!

In [26]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
 dropout_19 (Dropout)        multiple                  0 (unused)
                                                                 
Total params: 66955779 (255.42 MB)
Trainable params: 66955779 (255.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from transformers.keras_callbacks import PushToHubCallback
from transformers.keras_callbacks import KerasMetricCallback
import os
ACCESS_TOKEN = os.environ['HUGGINGFACE_HUB_ACCESS_CODE']
push_to_hub_callback = PushToHubCallback(
    output_dir="tt_abstract_classifier",
    tokenizer=tokenizer,
    hub_token=ACCESS_TOKEN
)

#metric_callback = KerasMetricCallback(metric_fn=compute_metrics)

In [ ]:
callbacks = [push_to_hub_callback]

In [30]:
model.fit(x=tf_train_set, epochs=10)

Epoch 1/10


2024-02-17 09:10:41.916924: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-02-17 09:10:42.495920: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp_10.


5/5 [==============================] - 45s 8s/step - loss: 1.0905
Epoch 2/10
5/5 [==============================] - 9s 2s/step - loss: 0.9642
Epoch 3/10
5/5 [==============================] - 10s 2s/step - loss: 0.8880
Epoch 4/10
5/5 [==============================] - 7s 2s/step - loss: 0.7704
Epoch 5/10
5/5 [==============================] - 5s 1s/step - loss: 0.6767
Epoch 6/10
5/5 [==============================] - 8s 942ms/step - loss: 0.6824
Epoch 7/10
5/5 [==============================] - 4s 757ms/step - loss: 0.6364
Epoch 8/10
5/5 [==============================] - 4s 891ms/step - loss: 0.5643
Epoch 9/10
5/5 [==============================] - 4s 839ms/step - loss: 0.4871
Epoch 10/10
5/5 [==============================] - 8s 2s/step - loss: 0.4736


In [32]:
tokenized = tokenizer(["some new text test text"])

In [33]:
pred = model.predict(tokenized['input_ids'])

1/1 [==============================] - 4s 4s/step


In [34]:
import numpy as np

In [35]:
pred_labels = np.argmax(pred.logits, axis=1)

In [38]:
id2label[pred_labels[0]]

'TT5'

In [51]:
#store classes in the classes file
import json
with open('../resources/label2id.json', 'w') as classes_file:
    json.dump(label2id, classes_file)